In [1]:
import pandas as pd
import numpy as np
from joblib import Memory
import time
import os
import pickle
import keras
# from TINTOlib.tinto import TINTO
from tensorflow_addons.metrics import RSquare

c:\Users\Medion\AppData\local\Programs\Python\Python311\Lib\site-packages\tensorflow_addons\utils\tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


In [ ]:
import numpy as np
import pandas as pd
import os
import gc

# Dimensional reduction classes
from sklearn.manifold import TSNE
# from tsnecuda import TSNE
from sklearn.decomposition import PCA

# Sklearn
from sklearn.preprocessing import MinMaxScaler

# Graphic library
import matplotlib
import matplotlib.image

# Additional libraries
import math
import pickle

###########################################################
################    TINTO EXECUTION    ####################
###########################################################



class TINTO:
    ###### default values ###############
    default_problem = "supervised"  # Define the type of dataset [supervised, unsupervised, regression]
    default_algorithm = "PCA"  # Dimensionality reduction algorithm (PCA o t-SNE)
    default_pixeles = 20  # Image's Pixels (one side)
    default_submatrix = True #Use or not use submatrix

    default_blur = False  # Active option blurring
    default_amplification = np.pi  # Amplification in blurring
    default_distance = 2  # Distance in blurring (number of pixels)
    default_steps = 4  # Steps in blurring
    default_option = 'mean'  # Option in blurring (mean and maximum)

    default_train_m = True
    default_seed = 20  # Seed
    default_times = 4  # Times replication in t-SNE
    default_verbose = False  # Verbose: if it's true, show the compilation text

    # A parte funciones
    default_save = False  # Save configurations (to reuse)
    default_load = False  # Load configurations (.pkl)

    def __init__(self, problem=default_problem,algorithm=default_algorithm, pixels=default_pixeles,submatrix=default_submatrix, blur=default_blur,
                 amplification=default_amplification, distance=default_distance, steps=default_steps, option=default_option,
                 seed=default_seed, times=default_times, train_m=default_train_m, verbose=default_verbose):
        self.problem = problem
        self.algorithm = algorithm
        self.pixels = pixels
        self.submatrix = submatrix

        self.blur = blur
        self.amplification = amplification
        self.distance = distance
        self.steps = steps
        self.option = option

        self.train_m = train_m
        self.seed = seed
        self.times = times
        self.verbose = verbose

        #self.src_data = src_data  # Source location (tidy data in csv without head)
        #self.dest_folder = dest_folder  # Destination location (folder)
        #src_data=None, dest_folder=None,

        self.error_pos = False  # Indicates the overlap of characteristic pixels.

    def saveHyperparameters(self, filename='objs'):
        """
        This function allows SAVING the transformation options to images in a Pickle object.
        This point is basically to be able to reproduce the experiments or reuse the transformation
        on unlabelled data.
        """
        with open(filename+".pkl", 'wb') as f:
            pickle.dump(self.__dict__, f)
        if self.verbose:
            print("It has been successfully saved in " + filename)

    def loadHyperparameters(self, filename='objs.pkl'):
        """
        This function allows LOADING the transformation options to images in a Pickle object.
        This point is basically to be able to reproduce the experiments or reuse the transformation
        on unlabelled data.
        """
        with open(filename, 'rb') as f:
            variables = pickle.load(f)
            self.algorithm = variables["algorithm"]
            self.pixels = variables["pixels"]

            self.blur = variables["blur"]
            self.amplification = variables["amplification"]
            self.distance = variables["distance"]
            self.steps = variables["steps"]
            self.option = variables["option"]

            self.seed = variables["seed"]
            self.times = variables["times"]
            self.verbose = variables["verbose"]

            #self.src_data = variables["src_data"]  # Source location (tidy data in csv without head)
            #self.dest_folder = variables["dest_folder"]  # Destination location (folder)

        if self.verbose:
            print("It has been successfully loaded in " + filename)


    def __square(self, coord):
        """
        This functionhas the purpose of being able to create the square delimitation of the resulting image.
        The steps it performs in the order of the code are as follows:
            - Calculate the average of the points $(x,y)$.
            - Centres the data at the point $(0,0)$.
            - Calculate the distance from the centre to the points.
            - The larger distance of \texttt{dista}, transforms it to integer.
            - Calculate the vertices of the square.
            - Move the points to quadrant $4$.
            - Transfers the vertices as well.
            - Returns the values, coordinates, and vertices.
        """
        m = np.mean(coord, axis=0).reshape((1, 2))
        coord_new = coord - m
        dista = (coord_new[:, 0] ** 2 + coord_new[:, 1] ** 2) ** 0.5
        maxi = math.ceil(max(dista))
        vertices = np.array([[-maxi, maxi], [-maxi, -maxi], [maxi, -maxi], [maxi, maxi]])
        coord_new = coord_new - vertices[0]
        vertices = vertices - vertices[0]
        return coord_new, vertices

    def __m_imagen(self, coord, vertices):
        """
        This function obtain the coordinates of the matrix. This function has
        the following specifications:
            - Create a matrix of coordinates and vertices.
            - Transform the coordinates into indices for the matrix.
            - Fill in the positions of the features.
            - Finally, a conditional is created if the features were grouped
              in the same position.
        """
        pixels = self.pixels
        size = (pixels, pixels)
        matrix = np.zeros(size)

        coord_m = (coord / vertices[2, 0]) * (pixels - 1)
        coord_m = np.round(abs(coord_m))

        for i, j in zip(coord_m[:, 1], coord_m[:, 0]):
            matrix[int(i), int(j)] = 1

        if np.count_nonzero(matrix != 0) != coord.shape[0]:
            return coord_m, matrix, True
        else:
            return coord_m, matrix, False

    def __createFilter(self):
        """
        In this function a filter is created since a matrix of size "2*distance*total_steps+1"
        is being created to act as a "filter", which covers the whole circular space of the minutiae
        determined by the distance and by the total number of steps.
        This "filter", which is a matrix, would be multiplied with a scalar, which is the intensity value.
        Finally, this resulting matrix is placed as a submatrix within the final matrix where the centre
        of the submatrix would be the position of the characteristic pixel.
        """
        distance = self.distance
        steps = self.steps
        amplification = self.amplification
        size_filter = int(2 * distance * steps + 1)
        center_x = distance * steps
        center_y = distance * steps
        filter = np.zeros([size_filter, size_filter])

        for step in reversed(range(steps)):
            r_actual = int(distance * (step + 1))  # current radius from largest to smallest

            # Function of intensity
            intensity = min(amplification * 1 / (np.pi * r_actual ** 2), 1)

            # Delimitation of the area
            lim_inf_i = max(center_x - r_actual - 1, 0)
            lim_sup_i = min(center_x + r_actual + 1, size_filter)
            lim_inf_j = max(center_y - r_actual - 1, 0)
            lim_sup_j = min(center_y + r_actual + 1, size_filter)

            # Allocation of values
            for i in range(lim_inf_i, lim_sup_i):
                for j in range(lim_inf_j, lim_sup_j):
                    if (center_x - i) ** 2 + (center_y - j) ** 2 <= r_actual ** 2:
                        filter[i, j] = intensity
        filter[center_x, center_y] = 1
        return filter

    def __blurringFilterSubmatrix(self, matrix, filter, values, coordinates):
        """
       This function is to be able to add more ordered contextual information to the image through the
       classical painting technique called blurring. This function develops the following main steps:
       - Take the coordinate matrix of the characteristic pixels.
       - Create the blurring according to the number of steps taken in a loop with the
         following specifications:
            - Delimit the blurring area according to $(x,y)$ on an upper and lower boundary.
            - Set the new intensity values in the matrix, taking into account that if there is
              pixel overlap, the maximum or average will be taken as specified.
        """
        option = self.option

        iter_values = iter(values)
        size_matrix = matrix.shape[0]
        size_filter = filter.shape[0]
        matrix_extended = np.zeros([size_filter + size_matrix, size_filter + size_matrix])
        matrix_add = np.zeros([size_filter + size_matrix, size_filter + size_matrix])
        center_filter = int((size_filter - 1) / 2)
        for i, j in coordinates:
            i = int(i)
            j = int(j)
            value = next(iter_values)
            submatrix = filter * value

            # Delimitación del área
            lim_inf_i = i
            lim_sup_i = i + 2 * center_filter + 1
            lim_inf_j = j
            lim_sup_j = j + 2 * center_filter + 1

            if option == 'mean':
                matrix_extended[lim_inf_i:lim_sup_i, lim_inf_j:lim_sup_j] += submatrix
                matrix_add[lim_inf_i:lim_sup_i, lim_inf_j:lim_sup_j] += (submatrix > 0) * 1
            elif option == 'maximum':
                matrix_extended[lim_inf_i:lim_sup_i, lim_inf_j:lim_sup_j] = np.maximum(
                    matrix_extended[lim_inf_i:lim_sup_i, lim_inf_j:lim_sup_j], submatrix)

        if option == 'mean':
            matrix_add[matrix_add == 0] = 1
            matrix_extended = matrix_extended / matrix_add

        matrix_final = matrix_extended[center_filter:-center_filter - 1, center_filter:-center_filter - 1]

        return matrix_final

    def __blurringFilter(self, matrix, coordinates):
        """
       This function is to be able to add more ordered contextual information to the image through the
       classical painting technique called blurring. This function develops the following main steps:
       - Take the coordinate matrix of the characteristic pixels.
       - Create the blurring according to the number of steps taken in a loop with the
         following specifications:
            - Delimit the blurring area according to $(x,y)$ on an upper and lower boundary.
            - Set the new intensity values in the matrix, taking into account that if there is
              pixel overlap, the maximum or average will be taken as specified.
        """

        x = int(coordinates[1])
        y = int(coordinates[0])
        valor_central = matrix[x, y]

        for p in range(self.steps):
            r_actual = int(matrix.shape[0] * self.distance * (p + 1))  # radio actual  de mayor a menor

            # Funcion de intensidad
            intensidad = min(self.amplification * valor_central / (np.pi * r_actual ** 2), valor_central)

            # Delimitación del área
            lim_inf_i = max(x - r_actual - 1, 0)
            lim_sup_i = min(x + r_actual + 1, matrix.shape[0])
            lim_inf_j = max(y - r_actual - 1, 0)
            lim_sup_j = min(y + r_actual + 1, matrix.shape[1])

            # Colocación de valores
            for i in range(lim_inf_i, lim_sup_i):
                for j in range(lim_inf_j, lim_sup_j):
                    if ((x - i) ** 2 + (y - j) ** 2 <= r_actual ** 2):
                        if (matrix[i, j] == 0):
                            matrix[i, j] = intensidad
                        elif (x != i and y != j):  # Sobreposición
                            if (self.option == 'mean'):
                                matrix[i, j] = (matrix[i, j] + intensidad) / 2
                            elif (self.option == 'maximum'):
                                matrix[i, j] = max(matrix[i, j], intensidad)
        return matrix


    def __saveSupervised(self, y, i, matrix_a):
        extension = 'png'  # eps o pdf
        subfolder = str(y).zfill(2)  # subfolder for grouping the results of each class
        name_image = str(i).zfill(6)
        route = os.path.join(self.folder, subfolder)
        route_complete = os.path.join(route, name_image + '.' + extension)
        # Subfolder check
        if not os.path.isdir(route):
            try:
                os.makedirs(route)
            except:
                print("Error: Could not create subfolder")

        matplotlib.image.imsave(route_complete, matrix_a, cmap='binary', format=extension)

        route_relative = os.path.join(subfolder, name_image+ '.' + extension)
        return route_relative

    def __saveRegressionOrUnsupervised(self, i, matrix_a):
        extension = 'png'  # eps o pdf
        subfolder = "images"
        name_image = str(i).zfill(6) + '.' + extension
        route = os.path.join(self.folder, subfolder)
        route_complete = os.path.join(route, name_image)
        if not os.path.isdir(route):
            try:
                os.makedirs(route)
            except:
                print("Error: Could not create subfolder")
        matplotlib.image.imsave(route_complete, matrix_a, cmap='binary', format=extension)

        route_relative = os.path.join(subfolder, name_image)
        return route_relative

    def __imageSampleFilterSubmatrix(self, X, Y, coord, matrix):
        """
        This function creates the samples, i.e., the images. This function has the following specifications:
        - The first conditional performs the pre-processing of the images by creating the matrices.
        - Then the for loop generates the images for each sample. Some assumptions have to be taken into
          account in this step:
            - The samples will be created according to the number of targets. Therefore, each folder that is
              created will contain the images created for each target.
            - In the code, the images are exported in PNG format; this can be changed to any other format.
        """
        # Hyperparams
        amplification = self.amplification
        distance = self.distance
        steps = self.steps
        option = self.option
        imagesRoutesArr=[]

        # Generate the filter
        if distance * steps * amplification != 0:  # The function is only called if there are no zeros (blurring).
            filter = self.__createFilter()

        total = X.shape[0]
        # In this part, images are generated for each sample.
        for i in range(total):
            matrix_a = np.zeros(matrix.shape)
            if distance * steps * amplification != 0:  # The function is only called if there are no zeros (blurring).
                matrix_a = self.__blurringFilterSubmatrix(matrix_a, filter, X[i], coord)
            else:  # (no blurring)
                iter_values_X = iter(X[i])
                for eje_x, eje_y in coord:
                    matrix_a[int(eje_x), int(eje_y)] = next(iter_values_X)

            if self.problem == "supervised":
                route=self.__saveSupervised(Y[i],i,matrix_a)
                imagesRoutesArr.append(route)
            elif self.problem == "unsupervised" or self.problem == "regression":
                route = self.__saveRegressionOrUnsupervised( i, matrix_a)
                imagesRoutesArr.append(route)
            else:
                print("Wrong problem definition. Please use 'supervised', 'unsupervised' or 'regression'")

            #Verbose
            if self.verbose:
                print("Created ", str(i + 1), "/", int(total))

        if self.problem == "supervised":
            data = {'images': imagesRoutesArr, 'class': Y}
            supervisedCSV = pd.DataFrame(data=data)
            supervisedCSV.to_csv(self.folder + "/supervised.csv", index=False)
        elif self.problem == "unsupervised":
            data = {'images': imagesRoutesArr}
            unsupervisedCSV = pd.DataFrame(data=data)
            unsupervisedCSV.to_csv(self.folder + "/unsupervised.csv", index=False)
        elif self.problem == "regression":
            data = {'images': imagesRoutesArr, 'values': Y}
            regressionCSV = pd.DataFrame(data=data)
            regressionCSV.to_csv(self.folder + "/regression.csv", index=False)

        return matrix

    def __imageSampleFilter(self, X, Y, coord, matrix):
        """
        This function creates the samples, i.e., the images. This function has the following specifications:
        - The first conditional performs the pre-processing of the images by creating the matrices.
        - Then the for loop generates the images for each sample. Some assumptions have to be taken into
          account in this step:
            - The samples will be created according to the number of targets. Therefore, each folder that is
              created will contain the images created for each target.
            - In the code, the images are exported in PNG format; this can be changed to any other format.
        """
        imagesRoutesArr = []
        total = X.shape[0]
        # Aquí es donde se realiza el preprocesamiento siendo 'matriz' = 'm'
        if self.train_m:

            matrix_a = np.zeros(matrix.shape)

            for i in range(X.shape[0]):
                for j in range(X.shape[1]):
                    matrix_a[int(coord[j, 1]), int(coord[j, 0])] = X[i, j]
                    matrix_a = self.__blurringFilter(matrix_a, coord[j] )

            matriz = np.copy(matrix_a)
        if self.verbose:
            print("Generating images......")
        # En esta parte se generan las imágenes por cada muestra
        for i in range(total):
            matrix_a = np.copy(matriz)

            for j in range(X.shape[1]):
                matrix_a[int(coord[j, 1]), int(coord[j, 0])] = X[i, j]
                matrix_a = self.__blurringFilter(matrix_a, coord[j])

            for j in range(X.shape[1]):
                matrix_a[int(coord[j, 1]), int(coord[j, 0])] = X[i, j]

            if self.problem == "supervised":
                    route=self.__saveSupervised(Y[i],i,matrix)
                    imagesRoutesArr.append(route)
            elif self.problem == "unsupervised" or self.problem == "regression":
                    route = self.__saveRegressionOrUnsupervised( i, matrix_a)
                    imagesRoutesArr.append(route)
            else:
                    print("Wrong problem definition. Please use 'supervised', 'unsupervised' or 'regression'")

            #Verbose
            if self.verbose:
                print("Created ", str(i + 1), "/", int(total))

        if self.problem == "supervised":
            data = {'images': imagesRoutesArr, 'class': Y}
            supervisedCSV = pd.DataFrame(data=data)
            supervisedCSV.to_csv(self.folder + "/supervised.csv", index=False)
        elif self.problem == "unsupervised":
            data = {'images': imagesRoutesArr}
            unsupervisedCSV = pd.DataFrame(data=data)
            unsupervisedCSV.to_csv(self.folder + "/unsupervised.csv", index=False)
        elif self.problem == "regression":
            data = {'images': imagesRoutesArr, 'values': Y}
            regressionCSV = pd.DataFrame(data=data)
            regressionCSV.to_csv(self.folder + "/regression.csv", index=False)

        return matrix

    def __obtainCoord(self, X):
        """
        This function uses the dimensionality reduction algorithm in order to represent the characteristic
        pixels in the image. The specifications of this function are:
        - Perform a normalisation of (0,1) to be able to represent the pixels inside the square.
        - Transpose the matrix.
        - Set the dimensionality reduction algorithm, PCA or t-SNE.
        """

        self.min_max_scaler = MinMaxScaler()
        X = self.min_max_scaler.fit_transform(X)

        labels = np.arange(X.shape[1])
        X_trans = X.T

        if self.verbose:
            print("Selected algorithm: " + self.algorithm)

        if self.algorithm == 'PCA':
            X_embedded = PCA(n_components=2, random_state=self.seed).fit(X_trans).transform(X_trans)
        elif self.algorithm == 't-SNE':
            for i in range(self.times):
                X_trans = np.append(X_trans, X_trans, axis=0)
                labels = np.append(labels, labels, axis=0)
            X_embedded = TSNE(n_components=2, random_state=self.seed, perplexity=50).fit_transform(X_trans)
        else:
            print("Error: Incorrect algorithm")
            X_embedded = np.random.rand(X.shape[1], 2)

        data_coord = {'x': X_embedded[:, 0], 'y': X_embedded[:, 1], 'Label': labels}
        dc = pd.DataFrame(data=data_coord)
        self.obtain_coord = dc.groupby('Label').mean().values

        del X_trans
        gc.collect()

    def __areaDelimitation(self):
        """
        This function performs the delimitation of the area
        """
        self.initial_coordinates, self.vertices = self.__square(self.obtain_coord)

    def __matrixPositions(self):
        """
        This function gets the positions in the matrix
        """
        self.pos_pixel_caract, self.m, self.error_pos = self.__m_imagen(self.initial_coordinates, self.vertices)

    def __createImage(self, X, Y, folder='prueba/'):
        """
        This function creates the images that will be processed by CNN.
        """

        X_scaled = self.min_max_scaler.transform(X)
        Y = np.array(Y)
        try:
            os.mkdir(folder)
            if self.verbose:
                print("The folder was created " + folder + "...")
        except:
            if self.verbose:
                print("The folder " + folder + " is already created...")

        if self.submatrix:
            self.m = self.__imageSampleFilterSubmatrix(X_scaled, Y, self.pos_pixel_caract, self.m)
        else:
            self.m = self.__imageSampleFilter(X_scaled, Y, self.pos_pixel_caract, self.m)

    def __trainingAlg(self, X, Y, folder='img_train/'):
        """
        This function uses the above functions for the training.
        """
        self.__obtainCoord(X)
        print("Hola")
        self.__areaDelimitation()
        self.__matrixPositions()

        self.__createImage(X, Y, folder)

    def __testAlg(self, X, Y=None, folder='img_test/'):
        """
        This function uses the above functions for the validation.
        """
        if (Y is None):
            Y = np.zeros(X.shape[0])
        self.__createImage(X, Y, folder)

    ###########################################################

    def generateImages(self,data, folder="/tintoData"):
        """
            This function generate and save the synthetic images in folders.
                - data : data CSV or pandas Dataframe
                - folder : the folder where the images are created
        """
        # Blurring verification

        if not self.blur:
            self.amplification = 0
            self.distance = 2
            self.steps = 0

        # Read the CSV
        self.folder = folder
        if type(data)==str:
            dataset = pd.read_csv(data)
            array = dataset.values
        elif isinstance(data, pd.DataFrame):
            array = data.values

        X = array[:, :-1]
        Y = array[:, -1]


        # Training
        self.__trainingAlg(X, Y, folder=folder)

        if self.verbose: print("End")


In [2]:
# Cargamos el dataset
df = pd.read_csv("C:/Users/Medion/Desktop/WEBOTS PROYECTOS/TFG/Data/ULA_8.csv")

In [3]:
print(df.shape)
print(df.iloc[:2,:])

(252004, 1602)
   Antena1Portadora1Modulo  Antena1Portadora1Angulo  Antena1Portadora2Modulo  \
0                 0.601309                -0.428778                 0.620221   
1                 0.436977                -0.835988                 0.447176   

   Antena1Portadora2Angulo  Antena1Portadora3Modulo  Antena1Portadora3Angulo  \
0                -0.449564                 0.626025                -0.508311   
1                -0.890598                 0.457048                -0.918748   

   Antena1Portadora4Modulo  Antena1Portadora4Angulo  Antena1Portadora5Modulo  \
0                 0.641815                -0.550858                 0.642635   
1                 0.475890                -0.948638                 0.477857   

   Antena1Portadora5Angulo  ...  Antena8Portadora97Modulo  \
0                -0.564398  ...                  0.859801   
1                -1.000932  ...                  0.630989   

   Antena8Portadora97Angulo  Antena8Portadora98Modulo  \
0                 -0.

In [4]:
# Fila que queremos predecir (falta añadirle ruido)
data = df.loc[(df['PosicionX'] == -1052) & (df['PosicionY'] == 2110)]
print(data)

        Antena1Portadora1Modulo  Antena1Portadora1Angulo  \
246511                 0.178537                -2.965653   

        Antena1Portadora2Modulo  Antena1Portadora2Angulo  \
246511                 0.162474                -2.972491   

        Antena1Portadora3Modulo  Antena1Portadora3Angulo  \
246511                 0.174036                -2.983824   

        Antena1Portadora4Modulo  Antena1Portadora4Angulo  \
246511                 0.177337                -3.009041   

        Antena1Portadora5Modulo  Antena1Portadora5Angulo  ...  \
246511                 0.171919                -3.118869  ...   

        Antena8Portadora97Modulo  Antena8Portadora97Angulo  \
246511                  0.727119                  2.574451   

        Antena8Portadora98Modulo  Antena8Portadora98Angulo  \
246511                  0.728335                  2.581856   

        Antena8Portadora99Modulo  Antena8Portadora99Angulo  \
246511                  0.746676                  2.633478   

        An

In [5]:
df_copia = df.copy()

In [6]:
# * Normalizamos los datos
df = df.head(100) # Limitamos el dataset a 100 filas para que no tarde tanto
columns_to_normalize = df.columns[:-2]
print(columns_to_normalize)
df_normalized = (df[columns_to_normalize] - df[columns_to_normalize].min()) / (df[columns_to_normalize].max() - df[columns_to_normalize].min())
print("-------")
print(df_normalized)
df_normalized = pd.concat([df_normalized, df[df.columns[-2]], df[df.columns[-1]]], axis=1)
print("-------")
print(df_normalized)

Index(['Antena1Portadora1Modulo', 'Antena1Portadora1Angulo',
       'Antena1Portadora2Modulo', 'Antena1Portadora2Angulo',
       'Antena1Portadora3Modulo', 'Antena1Portadora3Angulo',
       'Antena1Portadora4Modulo', 'Antena1Portadora4Angulo',
       'Antena1Portadora5Modulo', 'Antena1Portadora5Angulo',
       ...
       'Antena8Portadora96Modulo', 'Antena8Portadora96Angulo',
       'Antena8Portadora97Modulo', 'Antena8Portadora97Angulo',
       'Antena8Portadora98Modulo', 'Antena8Portadora98Angulo',
       'Antena8Portadora99Modulo', 'Antena8Portadora99Angulo',
       'Antena8Portadora100Modulo', 'Antena8Portadora100Angulo'],
      dtype='object', length=1600)
-------
    Antena1Portadora1Modulo  Antena1Portadora1Angulo  Antena1Portadora2Modulo  \
0                  0.662441                 0.430854                 0.681627   
1                  0.445998                 0.364748                 0.467091   
2                  0.496891                 0.450336                 0.514340   

In [28]:
from TINTOlib.tinto import TINTO
# Las imágenes se generan con el df. (no el normalizado)
image_folder = "C:/Users/Medion/Desktop/WEBOTS PROYECTOS/TFG/Images/Jupyter"
image_model = TINTO(problem = "regression", pixels = 35, blur = True)

if not os.path.exists(image_folder):
    print("Generating images")
    image_model.generateImages(df.iloc[:,:-1], image_folder)

# Store the model
print(image_model._TINTO__testAlg(df))
pickle.dump(image_model, open("C:/Users/Medion/Desktop/WEBOTS PROYECTOS/TFG/Images/Jupyter/image_model.pkl", "wb"))

img_paths = os.path.join(image_folder + "/regression.csv")

imgs = pd.read_csv(img_paths)
print(imgs.shape)
print(imgs.iloc[:5,:])

AttributeError: 'TINTO' object has no attribute 'min_max_scaler'

In [8]:
imgs["images"] = image_folder + "/" + imgs["images"]
imgs["images"] = imgs["images"].str.replace("\\","/")
# print(imgs["images"]) #   C:/Users/Medion/Desktop/WEBOTS PROYECTOS/TFG/Images/Jupyter/images/000000.png

# print("-------")
img_1 = imgs.iloc[-1:,:]
print(img_1["images"])

data_1 = df_normalized.iloc[-1:,:]
#print(data_1.shape)
# print("-------")
# print(data_1)

combined_data_img = pd.concat([img_1, data_1.iloc[:,:-1]], axis=1)
# print("-----")
# print(combined_data_img)

99    C:/Users/Medion/Desktop/WEBOTS PROYECTOS/TFG/I...
Name: images, dtype: object


In [9]:
combined_dataset_x = pd.concat([imgs,df_normalized.iloc[:,:-1]],axis=1)
print(combined_dataset_x.shape)
print(combined_dataset_x.iloc[:5,:]) # images, values, datos, posicionX

print("------")

combined_dataset_y = pd.concat([imgs,pd.concat([df_normalized.iloc[:,:-2], df_normalized.iloc[:,-1:]],axis=1)],axis=1)
print(combined_dataset_y.shape)
print(combined_dataset_y.iloc[:5,:]) # images, values, datos, posicionY

(100, 1603)
                                              images  values  \
0  C:/Users/Medion/Desktop/WEBOTS PROYECTOS/TFG/I...   302.0   
1  C:/Users/Medion/Desktop/WEBOTS PROYECTOS/TFG/I...   722.0   
2  C:/Users/Medion/Desktop/WEBOTS PROYECTOS/TFG/I...  1117.0   
3  C:/Users/Medion/Desktop/WEBOTS PROYECTOS/TFG/I...  -892.0   
4  C:/Users/Medion/Desktop/WEBOTS PROYECTOS/TFG/I... -1217.0   

   Antena1Portadora1Modulo  Antena1Portadora1Angulo  Antena1Portadora2Modulo  \
0                 0.662441                 0.430854                 0.681627   
1                 0.445998                 0.364748                 0.467091   
2                 0.496891                 0.450336                 0.514340   
3                 0.242174                 0.363422                 0.292943   
4                 0.285156                 0.904383                 0.301483   

   Antena1Portadora2Angulo  Antena1Portadora3Modulo  Antena1Portadora3Angulo  \
0                 0.431355                

In [10]:
df_x = combined_dataset_x.drop("PosicionX",axis=1).drop("values",axis=1) # Solo imagenes y datos
print(df_x.iloc[:1,:])

print("------")

df_y_for_x = combined_dataset_x["values"]
df_y_for_y = combined_dataset_y["PosicionY"]

print(df_y_for_x.iloc[:6]) # Posiciones de X
print("------")
print(df_y_for_y.iloc[:6]) # Posiciones de Y

                                              images  Antena1Portadora1Modulo  \
0  C:/Users/Medion/Desktop/WEBOTS PROYECTOS/TFG/I...                 0.662441   

   Antena1Portadora1Angulo  Antena1Portadora2Modulo  Antena1Portadora2Angulo  \
0                 0.430854                 0.681627                 0.431355   

   Antena1Portadora3Modulo  Antena1Portadora3Angulo  Antena1Portadora4Modulo  \
0                 0.673291                 0.417094                 0.675869   

   Antena1Portadora4Angulo  Antena1Portadora5Modulo  ...  \
0                 0.409565                 0.693136  ...   

   Antena8Portadora96Modulo  Antena8Portadora96Angulo  \
0                   0.52171                  0.453752   

   Antena8Portadora97Modulo  Antena8Portadora97Angulo  \
0                  0.536045                  0.449234   

   Antena8Portadora98Modulo  Antena8Portadora98Angulo  \
0                  0.513473                  0.447106   

   Antena8Portadora99Modulo  Antena8Portadora99An

In [11]:
# Mezclamos las filas

np.random.seed(64)
df_x = df_x.sample(frac=1).reset_index(drop=True) # 

np.random.seed(64)
df_y_for_x = df_y_for_x.sample(frac=1).reset_index(drop=True)

np.random.seed(64)
df_y_for_y = df_y_for_y.sample(frac=1).reset_index(drop=True)


In [12]:
trainings_size = 0.85                     # 85% training set
validation_size = 0.1                     # 10% validation set
test_size = 0.05                         # 5% test set

X_train = df_x.iloc[:int(trainings_size*len(df_x))]
print(X_train.iloc[:1,:])
y_train_x = df_y_for_x.iloc[:int(trainings_size*len(df_y_for_x))]
y_train_y = df_y_for_y.iloc[:int(trainings_size*len(df_y_for_y))]

X_val = df_x.iloc[int(trainings_size*len(df_x)):int((trainings_size+validation_size)*len(df_x))]
y_val_x = df_y_for_x.iloc[int(trainings_size*len(df_y_for_x)):int((trainings_size+validation_size)*len(df_y_for_x))]
y_val_y = df_y_for_y.iloc[int(trainings_size*len(df_y_for_y)):int((trainings_size+validation_size)*len(df_y_for_y))]

X_test = df_x.iloc[-int(test_size*len(df_x)):]
y_test_x = df_y_for_x.iloc[-int(test_size*len(df_y_for_x)):]
y_test_y = df_y_for_y.iloc[-int(test_size*len(df_y_for_y)):]

X_train_num = X_train.drop("images",axis=1)
X_val_num = X_val.drop("images",axis=1)
X_test_num = X_test.drop("images",axis=1)

                                              images  Antena1Portadora1Modulo  \
0  C:/Users/Medion/Desktop/WEBOTS PROYECTOS/TFG/I...                 0.416206   

   Antena1Portadora1Angulo  Antena1Portadora2Modulo  Antena1Portadora2Angulo  \
0                 0.506583                 0.431694                 0.512668   

   Antena1Portadora3Modulo  Antena1Portadora3Angulo  Antena1Portadora4Modulo  \
0                 0.441134                  0.49849                 0.425714   

   Antena1Portadora4Angulo  Antena1Portadora5Modulo  ...  \
0                 0.496221                 0.421953  ...   

   Antena8Portadora96Modulo  Antena8Portadora96Angulo  \
0                  0.222856                  0.932153   

   Antena8Portadora97Modulo  Antena8Portadora97Angulo  \
0                  0.205549                  0.931401   

   Antena8Portadora98Modulo  Antena8Portadora98Angulo  \
0                  0.225839                  0.932356   

   Antena8Portadora99Modulo  Antena8Portadora99An

In [119]:
import cv2
pixel = 35

X_train_img = np.array([cv2.resize(cv2.imread(img),(pixel,pixel)) for img in X_train["images"]])
X_val_img = np.array([cv2.resize(cv2.imread(img),(pixel,pixel)) for img in X_val["images"]])
X_test_img = np.array([cv2.resize(cv2.imread(img),(pixel,pixel)) for img in X_test["images"]])

# x_test_img_1 = np.array([cv2.resize(cv2.imread(img),(pixel,pixel)) for img in X_test["images"].iloc[0]])
# x_test_img_1 = np.array([cv2.resize(cv2.imread(img_1["images"]),(pixel,pixel))])
# x_test_img_1 = np.array([cv2.resize(cv2.imread(X_test["images"].iloc[:1]),(pixel,pixel))])

# route = img_1["images"]
# if isinstance(route, str):
#     print("AA")
# route_2 = X_test["images"].iloc[0]
# print(route_2)
# if isinstance(route_2, str):
#     print("BB")
    
# for img in X_test["images"].iloc[0]:
#     if isinstance(img, str):
#         print("CC")
#         break

# route = str(route)
# img = cv2.resize(cv2.imread(route), (pixel,pixel))
# print(img)

image_path = img_1["images"].iloc[0]
print(image_path)
image = cv2.imread(image_path)

if image is not None:
    # Continuar con el procesamiento de la imagen redimensionada
    try:
        resized_image = cv2.resize(image, (pixel, pixel))
        print(resized_image)
        print("---")
        print(np.array(resized_image))
        # Continuar con el procesamiento de la imagen redimensionada
    except Exception as e:
        print("Error al redimensionar la imagen:", e)
else:
    print("La imagen no se cargó correctamente.")


validacion_x = y_val_x
test_x = y_test_x
validacion_y = y_val_y
test_y = y_test_y

shape = len(X_train_num.columns) #1600

C:/Users/Medion/Desktop/WEBOTS PROYECTOS/TFG/Images/Jupyter/images/000099.png
[[[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 ...

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]]
---
[[[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [255 255

In [44]:
#predict([X_test_num, X_test_img])
print(X_test_num)
print("------")
print(X_test_img)

    Antena1Portadora1Modulo  Antena1Portadora1Angulo  Antena1Portadora2Modulo  \
95                 0.165156                 0.964502                 0.190093   
96                 0.181244                 0.045244                 0.232176   
97                 0.005502                 0.805736                 0.061883   
98                 0.063578                 0.127140                 0.128677   
99                 0.379075                 0.011871                 0.388504   

    Antena1Portadora2Angulo  Antena1Portadora3Modulo  Antena1Portadora3Angulo  \
95                 0.963395                 0.198283                 0.965993   
96                 0.037792                 0.234236                 0.041232   
97                 0.793160                 0.005682                 0.803624   
98                 0.132361                 0.064057                 0.105543   
99                 0.009860                 0.402047                 0.000000   

    Antena1Portadora4Modul

In [120]:
x = [-945.6397]
print(x[0])

-945.6397
